In [1]:
import os
import json

kaggle_token = {
  "username": "aziztlili",
  "key": "8eb4fb79b714b8f0c9ddc91d61920b28"
}

#Répertoire Kaggle correct sous Windows
kaggle_dir = os.path.expanduser("~/.kaggle")
os.makedirs(kaggle_dir, exist_ok=True)

#Sauvegarde dans le bon dossier
with open(os.path.join(kaggle_dir, "kaggle.json"), "w") as f:
    json.dump(kaggle_token, f)

#Permissions (peut être ignoré sous Windows sans erreur)
try:
    os.chmod(os.path.join(kaggle_dir, "kaggle.json"), 0o600)
except:
    pass  # Ignorer si chmod échoue sur Windows

print("Authentification configurée avec succès.")


Authentification configurée avec succès.


In [2]:
#!kaggle datasets download -d shanegerami/ai-vs-human-text
#!unzip ai-vs-human-text.zip

In [3]:
#Bibliotheques

In [4]:
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import Sequential, Input
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from keras.callbacks import EarlyStopping

In [5]:
#Charger et préparer les données

In [6]:
df = pd.read_csv("C:/Users/LOQ/Desktop/Summer Internship/balanced_1000_rows.csv", encoding="latin1")

# Extraire textes & labels existants
texts = df['text'].astype(str).tolist()
labels = df['generated'].tolist()

In [7]:
#Ajouter des courts textes humains
short_human_samples = [
    "aziz tlili",
    "hello",
    "ok",
    "yes",
    "no",
    "good job",
    "well done",
    "thanks",
    "maybe later",
    "not sure",
    "interesting",
    "cool",
    "fine",
    "why not",
    "good idea",
    "hi",
    "bye",
    "thank you"
]
short_labels = [0] * len(short_human_samples)


In [8]:
# Étendre le dataset
texts.extend(short_human_samples)
labels.extend(short_labels)

print(f" Après ajout : {Counter(labels)}")

 Après ajout : Counter({0.0: 518, 1.0: 500, nan: 1})


In [9]:
#Prétraitement NLP

In [10]:
#Tokenisation
max_words = 10000
max_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, maxlen=max_len, padding='post')

In [11]:
X = padded
y = labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
#Créer le modèle CNN

In [13]:
from keras import Sequential, Input
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

model = Sequential()
model.add(Input(shape=(max_len,)))
model.add(Embedding(input_dim=max_words, output_dim=64))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 300, 64)             │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 296, 64)             │          20,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 664,769 (2.54 MB)

 Trainable params: 664,769 (2.54 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
import numpy as np

# Convertir en tableau numpy
y_train = np.array(y_train, dtype=np.float32)

# Remplacer les NaN par une valeur valide, ici on peut choisir de les supprimer ou les fixer à 0 ou 1
# Exemple : on fixe à 0 si NaN
y_train = np.nan_to_num(y_train, nan=0.0)

# S’assurer que le type est cohérent : int ou float
# Si c’est une classification binaire, tu peux caster en int :
y_train = y_train.astype(np.int32)

# Vérifier qu’il n’y a plus de NaN
print("Y_train unique:", np.unique(y_train))

Y_train unique: [0 1]


In [15]:
#Entraîner le modèle

In [16]:
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[es]
)

Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 100ms/step - accuracy: 0.4990 - loss: 0.6961 - val_accuracy: 0.4724 - val_loss: 0.6874
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.6286 - loss: 0.6719 - val_accuracy: 0.7362 - val_loss: 0.6646
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - accuracy: 0.8548 - loss: 0.6122 - val_accuracy: 0.8037 - val_loss: 0.6082
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.9026 - loss: 0.5035 - val_accuracy: 0.8160 - val_loss: 0.4970
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.9286 - loss: 0.3282 - val_accuracy: 0.8589 - val_loss: 0.4096
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - accuracy: 0.9691 - loss: 0.1984 - val_accuracy: 0.8712 - val_loss: 0.3448
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.9935 - loss: 0.0962 - val_accuracy: 0.8773 - val_loss: 0.2821
Epoch 8/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.9962 - loss: 0.0509 - val_accuracy: 0.8773 - 

In [17]:
#Afficher quelques prédictions

In [18]:
def predict_text(text, tokenizer, model, max_len):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len)
    pred = model.predict(padded)[0][0]

    if pred >= 0.5:
        print(f"Prediction: AI-Generated (Confidence: {pred:.2f})")
    else:
        print(f"Prediction: Human (Confidence: {(1 - pred):.2f})")

# Exemple de test
predict_text("cocou salut bebe", tokenizer, model, max_len)
predict_text("This is an example of an AI generated text with more structure and longer form.", tokenizer, model, max_len)
predict_text("ama ena ya talee", tokenizer, model, max_len)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
Prediction: Human (Confidence: 0.97)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Prediction: Human (Confidence: 0.68)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Prediction: Human (Confidence: 0.96)


In [19]:
model.save("text_detector_model.keras")

In [20]:
pip install flask tensorflow

Note: you may need to restart the kernel to use updated packages.


In [21]:
import pickle

with open("tokenizer.pkl", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
